In [ ]:
!pip install emoji

In [ ]:
from google.colab import files
uploaded = files.upload() 

Saving glove.6B.50d.txt to glove.6B.50d (2).txt


In [ ]:
import numpy as np
from emo_utils import * # Pre written code to handle label to emoji and vice versa conversions and take care of global vectors
#Also contains code for backprop, one hot vector conversion etc. 
import emoji
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
#Getting data
x_train, y_train = read_csv('train_emoji.csv')
x_test, y_test = read_csv('tesss.csv')

In [ ]:
maxLen = len(max(x_train, key=len).split()) 

In [ ]:
#Printing examples
for idx in range(10):
    print(x_train[idx], label_to_emoji(y_train[idx]))

never talk to me again 😞
I am proud of your achievements 😄
It is the worst day in my life 😞
Miss you so much ❤️
food is life 🍴
I love you mum ❤️
Stop saying bullshit 😞
congratulations on your acceptance 😄
The assignment is too long  😞
I want to go play ⚾


In [ ]:
#One-hot vectors
y_oh_train = convert_to_one_hot(y_train, C = 5)
y_oh_test = convert_to_one_hot(y_test, C = 5)

In [ ]:
idx = 50
print(f"Sentence '{x_train[50]}' has label index {y_train[idx]}, which is emoji {label_to_emoji(y_train[idx])}", )
print(f"Label index {y_train[idx]} in one-hot encoding format is {y_oh_train[idx]}")

Sentence 'I missed you' has label index 0, which is emoji ❤️
Label index 0 in one-hot encoding format is [1. 0. 0. 0. 0.]


In [ ]:
word_to_index, index_to_word, word_to_vec_map = read_glove_vecs('glove.6B.50d.txt') #GloVe contains pre-trained word vectors
#More info about GloVe on https://nlp.stanford.edu/projects/glove/

In [ ]:
word="cucumber"
idx = 28984
print("the index of", word, "in the vocabulary is", word_to_index[word]) #Loading index of word as found in GloVe
print("the", str(idx) + "th word in the vocabulary is", index_to_word[idx]) #Loading word corresponding to given index

the index of cucumber in the vocabulary is 66229
the 28984th word in the vocabulary is ammunition


In [ ]:
def sentence_to_avg(sentence, word_to_vec_map):
    """
    Converts a sentence (string) into a list of words (strings). Extracts the GloVe representation of each word
    and averages its value into a single vector encoding the meaning of the sentence.
    
    """
    
    ### START CODE HERE ###
    # Step 1: Split sentence into list of lower case words (≈ 1 line)
    words = sentence.lower().split()

    # Initialize the average word vector, should have the same shape as your word vectors.
    avg = np.zeros(50,)
    
    # Step 2: average the word vectors. You can loop over the words in the list "words".
    total = 0
    for w in words:
        total += word_to_vec_map[w]
    avg = total / len(words)
    
    ### END CODE HERE ###
    
    return avg

In [ ]:
avg = sentence_to_avg("I love to eat good food", word_to_vec_map)
print("avg = \n", avg)

avg = 
 [ 0.23663833  0.14996283 -0.38402217 -0.46861167  0.75710333 -0.07693733
 -0.54478167 -0.0234841   0.0977245   0.27182826 -0.05338167  0.55890133
  0.08024833 -0.19970717  0.85871167  0.38698983  0.33208207  0.161245
 -0.0183219  -0.61215983  0.0089565   0.41839133  0.700015    0.27518367
  0.63120233 -1.543525   -0.65569167  0.07079     0.94365833 -0.54656333
  3.26546667  0.73507833 -0.47767333  0.2426375  -0.092845   -0.01952117
 -0.06060167  0.3481665   0.41174667 -0.2800495  -0.05578138  0.06780998
  0.008335    0.24058467  0.49523767  0.3724065  -0.231683   -0.38439783
  0.00644145  0.49646167]


In [ ]:
def model(X, Y, word_to_vec_map, learning_rate = 0.16, num_iterations = 500):
    """
    Model to train word vector representations in numpy.
    
    """
    
    np.random.seed(1)

    # Define number of training examples
    m = Y.shape[0]                          # number of training examples
    n_y = 5                                 # number of classes  
    n_h = 50                                # dimensions of the GloVe vectors 
    
    # Initialize parameters using Xavier initialization
    W = np.random.randn(n_y, n_h) / np.sqrt(n_h)
    b = np.zeros((n_y,))
    
    # Convert Y to Y_onehot with n_y classes
    Y_oh = convert_to_one_hot(Y, C = n_y) 
    
    # Optimization loop
    for t in range(num_iterations): # Loop over the number of iterations
        for i in range(m):          # Loop over the training examples
            
            ### START CODE HERE ### (≈ 4 lines of code)
            # Average the word vectors of the words from the i'th training example
            avg = sentence_to_avg(X[i], word_to_vec_map)

            # Forward propagate the avg through the softmax layer
            z = np.matmul(W,avg) + b
            a = softmax(z)

            # Compute cost using the i'th training label's one hot representation and "A" (the output of the softmax)
            cost = -sum(Y_oh[i] * np.log(a))
            ### END CODE HERE ###
            
            # Compute gradients 
            dz = a - Y_oh[i]
            dW = np.dot(dz.reshape(n_y,1), avg.reshape(1, n_h))
            db = dz

            # Update parameters with Stochastic Gradient Descent
            W = W - learning_rate * dW
            b = b - learning_rate * db

        if t % 100 == 0:
            print("Epoch: " + str(t) + " --- cost = " + str(cost))
            pred = predict(X, Y, W, b, word_to_vec_map) #predict is defined in emo_utils.py

    return pred, W, b

In [ ]:
print(x_train.shape)
print(y_train.shape)
print(np.eye(5)[y_train.reshape(-1)].shape)
print(x_train[0])
print(type(x_train))
y = np.asarray([5, 0, 0, 5, 4, 4, 4, 6, 6, 4, 1, 1, 5, 6, 6, 3, 6, 3, 4, 4])
print(y.shape)

x = np.asarray(['I am going to the bar tonight', 'I love you', 'miss you my dear',
 'Lets go party and drinks','Congrats on the new job','Congratulations',
 'I am so happy for you', 'Why are you feeling bad', 'What is wrong with you',
 'You totally deserve this prize', 'Let us go play football',
 'Are you down for football this afternoon', 'Work hard play harder',
 'It is suprising how people can be dumb sometimes',
 'I am very disappointed','It is the best day in my life',
 'I think I will end up alone','My life is so boring','Good job',
 'Great so awesome'])

print(x.shape)
print(np.eye(5)[y_train.reshape(-1)].shape)
print(type(x_train))

(132,)
(132,)
(132, 5)
never talk to me again
<class 'numpy.ndarray'>
(20,)
(20,)
(132, 5)
<class 'numpy.ndarray'>


In [ ]:
pred, W, b = model(x_train, y_train, word_to_vec_map)
print(pred)

Epoch: 0 --- cost = 3.9642655532122792
Accuracy: 0.6742424242424242
Epoch: 100 --- cost = 0.006958417613812643
Accuracy: 0.9696969696969697
Epoch: 200 --- cost = 0.0029234847954393074
Accuracy: 0.9696969696969697
Epoch: 300 --- cost = 0.0024962205201839067
Accuracy: 0.9772727272727273
Epoch: 400 --- cost = 0.00293940374423508
Accuracy: 0.9924242424242424
[[3.]
 [2.]
 [3.]
 [0.]
 [4.]
 [0.]
 [3.]
 [2.]
 [3.]
 [1.]
 [3.]
 [3.]
 [1.]
 [3.]
 [2.]
 [3.]
 [2.]
 [3.]
 [1.]
 [2.]
 [3.]
 [0.]
 [2.]
 [2.]
 [2.]
 [1.]
 [4.]
 [2.]
 [2.]
 [4.]
 [0.]
 [3.]
 [4.]
 [3.]
 [0.]
 [3.]
 [2.]
 [2.]
 [3.]
 [4.]
 [2.]
 [2.]
 [0.]
 [2.]
 [3.]
 [0.]
 [3.]
 [2.]
 [4.]
 [3.]
 [0.]
 [3.]
 [3.]
 [3.]
 [4.]
 [2.]
 [1.]
 [1.]
 [1.]
 [2.]
 [3.]
 [1.]
 [0.]
 [0.]
 [0.]
 [3.]
 [4.]
 [4.]
 [2.]
 [2.]
 [1.]
 [2.]
 [0.]
 [3.]
 [2.]
 [2.]
 [0.]
 [0.]
 [3.]
 [1.]
 [2.]
 [1.]
 [2.]
 [2.]
 [4.]
 [3.]
 [3.]
 [2.]
 [4.]
 [0.]
 [0.]
 [0.]
 [3.]
 [3.]
 [3.]
 [2.]
 [0.]
 [1.]
 [2.]
 [3.]
 [0.]
 [2.]
 [2.]
 [2.]
 [3.]
 [2.]
 [2.]
 

In [ ]:
print("Training set:")
pred_train = predict(x_train, y_train, W, b, word_to_vec_map)
print('Test set:')
pred_test = predict(x_test, y_test, W, b, word_to_vec_map)

Training set:
Accuracy: 1.0
Test set:
Accuracy: 0.9107142857142857


In [ ]:
X_my_sentences = np.array(["life is full of happy thoughts and it is great", "i love to eat food", "life is good", "lets play with a ball", "i am mad", "not feeling happy"])
Y_my_labels = np.array([[2], [2], [2], [1], [3],[3]])
#0-->heart, 1-->ball, 2-->smile, 3-->disappointed, 4-->fork and knife
pred = predict(X_my_sentences, Y_my_labels , W, b, word_to_vec_map)
print_predictions(X_my_sentences, pred)

Accuracy: 0.6666666666666666

life is full of happy thoughts and it is great 😄
i love to eat food 🍴
life is good 😄
lets play with a ball ⚾
i am mad 😞
not feeling happy 😄
